In [1]:
import numpy as np
import os
from keras.models import Model
import pandas as pd
import os
import matplotlib.pyplot
%matplotlib inline
import librosa
import lib.util as ut
import tensorflow as tf

Using TensorFlow backend.


## Load the model we want to make inferences with

In [2]:
model_path='./trained_models/model_paper/model_25epochs.h5'

## To divide audio files. When making inferences over whole audio files, we pass interval_step=sample_rate/2 to produce 50% overlapping chunks of 3 second audio

In [3]:
def divide_audio_file(path, intervals_seconds, interval_step, sample_rate=8000):
    """

    """
    # loads file and converts to the specified sample rate    
    audio, fs = librosa.load(path, sample_rate)
    
    audio_array = []
    for i in range(0, audio.shape[0], interval_step):   
        interval = audio[i:i+sample_rate*intervals_seconds]
#         print("interval from {} to {}".format(i, i+sample_rate*intervals_seconds))
        
        # if the last interval is shorter han the interval in seconds we define we are going to ignore it
        if interval.shape[0] < sample_rate*intervals_seconds:
            break
        else:
            if (not ut.is_silence(interval,thresold_samples=0.70)):
                audio_array.append(interval)
            else:
                print("Omitting chunk with silences in file {}".format(path))

    return np.array(audio_array)

## Like the original create_tf_records but creating numpy array's instead

In [4]:
def get_chunks_from_raw(audio_path, file_names_df, num_samples):
    
    x_data = []
    labels = []
    tf_record_files = 1  
    for file in os.listdir(audio_path):

        file_path = audio_path + "/" + file
        short_name = file.split(".")[0]

        # if the file is in the dataframe with the file names(train or test) we divide it, if not we ignore
        if short_name in file_names_df["0"].values:
            print("reading file {}".format(file))

            divided_file =  divide_audio_file(file_path, intervals_seconds, sample_rate)

            file_label = file_names_df[file_names_df["0"] == short_name]["class"].values[0]
            labels_array = np.ones(divided_file.shape[0]) * file_label
            
            x_data.extend(divided_file)
            labels.extend(labels_array)            
            
            if len(x_data) > num_samples:
                tf_record_files += 1
                break;
                        
        else:
            print("file {} not in the dataframe".format(file))
       
                 
    return (x_data,labels)



## makes inference of a whole audio by dividing it into overlapping (50%) chunks

In [5]:
def predict_one_audio(file_name,label):
    file_path = audio_path + "/" + file_name
    audio_chunks=divide_audio_file(file_path, intervals_seconds, int(sample_rate*1.5))
    audio_chunks = audio_chunks.reshape([audio_chunks.shape[0],intervals_seconds * sample_rate,1])
    labels_array = np.ones(audio_chunks.shape[0]) * label
    predictions = model.predict(audio_chunks)
#     print(labels_array.shape)
#     print(audio_chunks.shape)
#     vc = pd.DataFrame(predictions)[0].apply(lambda x: 0 if x < 0.5 else 1).value_counts()
#     if (vc[0]<vc[1]):
#         res = 1
#     else:
#         res = 0
        
    return (predictions)
    
    

## Get string labels from numeric ones to build a confusion matrix 

In [6]:
def getLabels(x):
    if (x>=0.5):
        label = "CY"
    else:
        label = "CN"
    return(label)
    


## Begin of the execution

In [7]:
from keras.models import load_model
model = load_model(model_path)
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:74: UserWarning: The `MaxoutDense` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `MaxoutDense` layer is deprecated '


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24000, 1)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24000, 64)         8320      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3000, 64)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 3000, 64, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3000, 64, 96)      5952      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 375, 64, 96)       0         
________________

In [8]:
model.input_shape[1]

24000

In [9]:
audio_path = "./data/audiodata"
labels_file = "./data/conflictlevel.csv"

intervals_seconds = 3
sample_rate = 8000

In [10]:
labels_df = pd.read_csv(labels_file, header=None)
labels_df["class"] = labels_df[1].apply(lambda x: 0 if x < 0 else 1)



In [11]:
# train_df = pd.read_csv("./data/train_files.csv")
test_df = pd.read_csv("./data/test_files.csv")
# validation_df = pd.read_csv("./data/validation_files.csv")

## Prediction on test set

In [12]:
test_df.head()

,0,1,class
0,06-05-10_2250_2280,7.9,1
1,06-09-13_2310_2340,8.5,1
2,07-05-16_1440_1470,-5.9,0
3,07-09-17_720_750,1.1,1
4,06-11-08_1410_1440,7.4,1


In [13]:
predictions_list = []

for file, cl in test_df[["0", "class"]].values:
    pred = predict_one_audio(file + ".wav",cl)
    
    print("predicting for file {}".format(file + ".wav"))
    predictions_list.append([i for sublist in pred for i in sublist])
    

predicting for file 06-05-10_2250_2280.wav
predicting for file 06-09-13_2310_2340.wav
predicting for file 07-05-16_1440_1470.wav
predicting for file 07-09-17_720_750.wav
predicting for file 06-11-08_1410_1440.wav
predicting for file 06-05-10_1500_1530.wav
predicting for file 06-12-13_1680_1710.wav
predicting for file 06-12-20_1410_1440.wav
predicting for file 08-01-09_1500_1530.wav
predicting for file 07-02-21_1770_1800.wav
predicting for file 07-02-28_390_420.wav
predicting for file 06-04-12_1110_1140.wav
predicting for file 06-11-08_720_750.wav
predicting for file 06-10-11_2040_2070.wav
predicting for file 06-10-18_570_600.wav
predicting for file 06-09-06_1260_1290.wav
predicting for file 08-01-23_1200_1230.wav
predicting for file 06-12-13_1620_1650.wav
predicting for file 06-12-13_270_300.wav
predicting for file 07-02-14_390_420.wav
predicting for file 07-01-31_1440_1470.wav
predicting for file 07-01-31_1470_1500.wav
predicting for file 06-10-18_1290_1320.wav
predicting for file 07-

predicting for file 06-11-22_1680_1710.wav
predicting for file 08-01-15_2220_2250.wav
predicting for file 06-12-20_1260_1290.wav
predicting for file 06-04-05_990_1020.wav
predicting for file 06-09-06_870_900.wav
predicting for file 08-01-23_1350_1380.wav
predicting for file 06-10-04_2190_2220.wav
predicting for file 06-04-12_1050_1080.wav
predicting for file 06-04-12_2430_2460.wav
predicting for file 06-11-29_1830_1860.wav
predicting for file 07-02-21_2160_2190.wav
predicting for file 07-06-20_570_600.wav
predicting for file 08-01-30_480_510.wav
predicting for file 07-09-17_1380_1410.wav
predicting for file 06-05-10_510_540.wav
predicting for file 07-12-19_1740_1770.wav
predicting for file 07-03-28_780_810.wav
predicting for file 06-09-13_1830_1860.wav
predicting for file 08-01-15_960_990.wav
predicting for file 07-03-28_1320_1350.wav
predicting for file 08-01-23_2280_2310.wav
predicting for file 07-02-28_450_480.wav
predicting for file 08-02-06_2070_2100.wav
predicting for file 07-05-

In [14]:
pred_df = pd.DataFrame({"prediction": predictions_list})

In [55]:
# test_df = test_df.iloc[:5]

In [15]:
final_df = test_df.join(pred_df)

In [16]:
final_df["avg"] = final_df["prediction"].apply(lambda x: np.mean(x))
final_df["stdv"] = final_df["prediction"].apply(lambda x: np.std(x))
final_df["positive"] = final_df["prediction"].apply(lambda x: sum(i >= 0.5 for i in x))
final_df["negative"] = final_df["prediction"].apply(lambda x: sum(i < 0.5 for i in x))
final_df["majority"] = final_df.apply(lambda row: 1 if row["positive"] >= row["negative"] else 0, axis=1)

In [17]:
final_df.head()

,0,1,class,prediction,avg,stdv,positive,negative,majority
0,06-05-10_2250_2280,7.9,1,"[0.99421763, 0.9979219, 0.99968576, 0.9975103,...",0.998532,0.001665,19,0,1
1,06-09-13_2310_2340,8.5,1,"[0.94004524, 0.7154069, 0.98633236, 0.99207413...",0.605716,0.372241,11,8,1
2,07-05-16_1440_1470,-5.9,0,"[0.009947419, 1.5199184e-06, 0.20875564, 0.000...",0.168271,0.319009,3,16,0
3,07-09-17_720_750,1.1,1,"[0.97027767, 0.48355103, 0.00017952919, 0.0045...",0.465564,0.401484,7,12,0
4,06-11-08_1410_1440,7.4,1,"[0.027306736, 0.31755626, 0.8729861, 0.9844569...",0.793733,0.325576,15,4,1


In [18]:
final_df.to_csv("./predictions_files.csv", index=False)

In [61]:
labelTr = np.vectorize(getLabels)
pre_labels = labelTr(res)
tru_labels = labelTr(y_arr)

In [62]:
from sklearn.metrics import classification_report
metrics = classification_report(tru_labels,pre_labels)
print(metrics)

             precision    recall  f1-score   support

         CN       0.60      1.00      0.75         3
         CY       0.00      0.00      0.00         2

avg / total       0.36      0.60      0.45         5



/home/alberto/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
